# Analyse Exploratoire - Fashion Store Sales

Ce notebook presente l'analyse exploratoire du jeu de donnees des ventes d'un site e-commerce de mode. L'objectif est d'examiner la structure du fichier, identifier les entites metier principales, et mettre en evidence les redondances et anomalies.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../data/fashion_store_sales.csv')
print(f"Dimensions : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head()

## 1. Structure du fichier

In [ ]:
df.info()

In [ ]:
summary = pd.DataFrame({
    'type': df.dtypes,
    'non_null': df.notna().sum(),
    'null': df.isna().sum(),
    'unique': df.nunique(),
    'exemple': df.iloc[0]
})
summary

## 2. Identification des entites metier

Le fichier CSV est un fichier plat (denormalise) qui contient plusieurs entites metier melangees dans chaque ligne. En analysant les cardinalites, on identifie :

In [ ]:
entities = {
    'Articles de vente (item_id)': df['item_id'].nunique(),
    'Ventes (sale_id)': df['sale_id'].nunique(),
    'Produits (product_id)': df['product_id'].nunique(),
    'Clients (customer_id)': df['customer_id'].nunique(),
    'Canaux (channel)': df['channel'].nunique(),
}
for k, v in entities.items():
    print(f"{k}: {v} valeurs uniques")

print(f"\nChaque ligne = 1 article de vente (grain du fichier)")
print(f"Nombre moyen d'articles par vente : {df.shape[0] / df['sale_id'].nunique():.1f}")

In [ ]:
items_per_sale = df.groupby('sale_id')['item_id'].count()
print("Distribution du nombre d'articles par vente :")
print(items_per_sale.value_counts().sort_index())

## 3. Analyse de redondance

Dans le fichier plat, les attributs client sont repetes a chaque ligne ou le client apparait. Idem pour les produits. C'est la redondance typique qu'on elimine par la normalisation.

In [ ]:
customer_cols = ['customer_id', 'first_name', 'last_name', 'email', 'gender', 'age_range', 'signup_date', 'country']
product_cols = ['product_id', 'product_name', 'category', 'brand', 'color', 'size', 'catalog_price', 'cost_price']

nb_customer_rows = df.shape[0]
nb_unique_customers = df['customer_id'].nunique()
nb_unique_products = df['product_id'].nunique()

print(f"Attributs client repetes en moyenne {nb_customer_rows / nb_unique_customers:.1f}x")
print(f"Attributs produit repetes en moyenne {nb_customer_rows / nb_unique_products:.1f}x")
print(f"\nSur 2253 lignes, seuls {nb_unique_customers} ensembles client et {nb_unique_products} ensembles produit sont distincts.")

## 4. Valeurs manquantes

In [ ]:
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)
missing_df = pd.DataFrame({'manquants': missing, 'pourcentage': missing_pct})
missing_df[missing_df['manquants'] > 0]

In [ ]:
empty_str = (df == '').sum()
for col in ['first_name', 'last_name', 'email', 'total_amount']:
    null_count = df[col].isna().sum()
    empty_count = (df[col].astype(str).str.strip() == '').sum() if null_count == 0 else null_count
    print(f"{col}: {empty_count} valeurs manquantes ou vides")

**Constat :** Environ 180 first_name, 224 email et 225 total_amount sont manquants. Le total_amount est derivable (somme des item_total par vente), donc ce n'est pas une perte de donnees. Les noms et emails manquants suggerent des clients partiellement anonymes.

## 5. Coherence de product_id

In [ ]:
product_attrs = ['product_name', 'category', 'brand', 'color', 'size', 'catalog_price', 'cost_price']
consistency = df.groupby('product_id')[product_attrs].nunique()
inconsistent = consistency[consistency.gt(1).any(axis=1)]

if inconsistent.empty:
    print("Tous les product_id sont coherents avec leurs attributs.")
else:
    print(f"{len(inconsistent)} product_id incoherents detectes :")
    print(inconsistent)

In [ ]:
customer_attrs = ['gender', 'age_range', 'signup_date', 'country']
cust_consistency = df.groupby('customer_id')[customer_attrs].nunique()
cust_inconsistent = cust_consistency[cust_consistency.gt(1).any(axis=1)]

if cust_inconsistent.empty:
    print("Tous les customer_id sont coherents avec leurs attributs.")
else:
    print(f"{len(cust_inconsistent)} customer_id incoherents :")
    print(cust_inconsistent)

## 6. Valeurs du domaine

In [ ]:
categorical = ['channel', 'channel_campaigns', 'category', 'brand', 'color', 'size', 'gender', 'age_range', 'country']

for col in categorical:
    print(f"\n--- {col} ({df[col].nunique()} valeurs) ---")
    print(df[col].value_counts())

**Observations :**
- Une seule marque (Tiva) et un seul genre (Female) dans tout le dataset
- 2 canaux avec mapping 1:1 vers les campagnes
- La colonne size melange tailles textiles (XS, S, M, L, XL) et pointures numeriques (35, 36, 38, 40)

## 7. Mapping channel / channel_campaigns

In [ ]:
channel_mapping = df[['channel', 'channel_campaigns']].drop_duplicates()
print("Mapping channel -> channel_campaigns :")
print(channel_mapping.to_string(index=False))
print(f"\nRelation 1:1 confirmee : chaque canal a exactement une campagne associee.")

## 8. Analyse des remises

In [ ]:
discounted = df[df['discount_applied'] > 0]
print(f"Lignes avec remise : {len(discounted)} sur {len(df)} ({len(discounted)/len(df)*100:.1f}%)")
print(f"\nValeurs distinctes de discount_percent :")
print(df['discount_percent'].value_counts())

## 9. Verification des formules de prix

In [ ]:
df['check_unit_price'] = (df['original_price'] - df['discount_applied']).round(2)
unit_price_ok = (df['check_unit_price'] == df['unit_price']).all()
print(f"unit_price = original_price - discount_applied : {unit_price_ok}")

df['check_item_total'] = (df['quantity'] * df['unit_price']).round(2)
item_total_ok = (df['check_item_total'] == df['item_total']).all()
print(f"item_total = quantity * unit_price : {item_total_ok}")

df['check_discounted'] = (df['discount_applied'] > 0).astype(int)
discounted_ok = (df['check_discounted'] == df['discounted']).all()
print(f"discounted = 1 si discount_applied > 0 : {discounted_ok}")

print("\nConclusion : tous les champs derives sont calculables a partir de original_price, discount_applied et quantity.")
print("Ils peuvent etre supprimes en DKNF et recalcules via une vue.")

df.drop(columns=['check_unit_price', 'check_item_total', 'check_discounted'], inplace=True)

## 10. Distributions statistiques

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

df['category'].value_counts().plot(kind='bar', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title('Repartition par categorie')
axes[0, 0].tick_params(axis='x', rotation=45)

df['country'].value_counts().plot(kind='bar', ax=axes[0, 1], color='coral')
axes[0, 1].set_title('Repartition par pays')
axes[0, 1].tick_params(axis='x', rotation=45)

df['channel'].value_counts().plot(kind='bar', ax=axes[1, 0], color='seagreen')
axes[1, 0].set_title('Repartition par canal')
axes[1, 0].tick_params(axis='x', rotation=0)

df['age_range'].value_counts().sort_index().plot(kind='bar', ax=axes[1, 1], color='mediumpurple')
axes[1, 1].set_title('Repartition par tranche d\'age')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df['original_price'].hist(bins=30, ax=axes[0], color='steelblue', edgecolor='white')
axes[0].set_title('Distribution des prix originaux')
axes[0].set_xlabel('Prix')

df['sale_date'] = pd.to_datetime(df['sale_date'])
daily_sales = df.groupby('sale_date')['sale_id'].nunique()
daily_sales.plot(ax=axes[1], color='coral')
axes[1].set_title('Nombre de ventes par jour')
axes[1].set_xlabel('Date')

plt.tight_layout()
plt.show()

## 11. Plage de dates

In [ ]:
print(f"Premiere date : {df['sale_date'].min()}")
print(f"Derniere date : {df['sale_date'].max()}")
print(f"Nombre de jours distincts : {df['sale_date'].nunique()}")

## 12. Synthese des anomalies et observations

| Observation | Detail |
|---|---|
| Valeurs manquantes | 180 first_name, 224 email, 225 total_amount |
| Marque unique | Toutes les lignes ont brand = Tiva |
| Genre unique | Toutes les lignes ont gender = Female |
| Tailles mixtes | Melange de tailles textiles et pointures numeriques |
| Mapping 1:1 channel/campaign | App Mobile -> App Mobile, E-commerce -> Website Banner |
| Remises limitees | Seulement 10% et 30%, appliquees sur 9.9% des lignes |
| Champs derivables | unit_price, discount_percent, discounted, item_total, total_amount |
| Redondance massive | Attributs client repetes ~4x, attributs produit ~4.5x |
| Ventes multi-articles | 905 ventes contiennent 2253 articles (2-3 articles par vente) |

Ces constats motivent directement la normalisation en 3NF puis DKNF pour eliminer la redondance et garantir l'integrite des donnees.